<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

In [1]:
import random
import numpy as np
from torchvision import datasets, transforms

In [2]:
!wget -O mnist.npz https://s3.amazonaws.com/img-datasets/mnist.npz

--2023-11-02 09:26:22--  https://s3.amazonaws.com/img-datasets/mnist.npz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.103.176, 52.217.119.16, 54.231.170.24, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.103.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11490434 (11M) [application/octet-stream]
Saving to: ‘mnist.npz’

mnist.npz           100%[===================>]  10.96M  40.0MB/s    in 0.3s    

2023-11-02 09:26:22 (40.0 MB/s) - ‘mnist.npz’ saved [11490434/11490434]



In [3]:
# Let's read the mnist dataset

def load_mnist(path='mnist.npz'):
    with np.load(path) as f:
        x_train, _y_train = f['x_train'], f['y_train']
        x_test, _y_test = f['x_test'], f['y_test']

    x_train = x_train.reshape(-1, 28 * 28) / 255.
    x_test = x_test.reshape(-1, 28 * 28) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1

    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

## Exercise 1

In this exercise your task is to fill in the gaps in this code by implementing the backpropagation algorithm
Once this is done, you can run the network on the MNIST example and see how it performs. Feel free to play with the parameters. Your model should achieve 90%+ accuracy after a few epochs.


## Exercise 2 (Optional)

Implement a "fully vectorized" version, i.e. one using matrix operations instead of going over examples one by one within a minibatch.


In [31]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z) * (1 - sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in zip(x_mini_batch, y_mini_batch):
            delta_nabla_b, delta_nabla_w = self.backprop(x.reshape(784,1), y.reshape(10,1))
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w - (eta / len(x_mini_batch)) * nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(x_mini_batch)) * nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.

        # First initialize the list of gradient arrays
        delta_nabla_b = []
        delta_nabla_w = []

        # Then go forward remembering all values before and after activations
        # in two other array list
        a = x
        v = x
        activations = [a]
        values = [a]
        for b, w in zip(self.biases, self.weights):
            v = np.dot(w, a) + b
            values.append(v)
            a = sigmoid(v)
            activations.append(a)

        # Now go backward from the final cost applying backpropagation
        next_layer_gradient = sigmoid_prime(v) * self.cost_derivative(a, y)
        reversed_zip = list(reversed(list(zip(self.biases, self.weights, activations[:-1], values[:-1]))))
        for b, w, activation, val in reversed_zip:
            delta_nabla_b.append(np.array(next_layer_gradient).reshape(-1, 1))
            delta_nabla_w.append(np.array(next_layer_gradient).reshape(-1, 1) @ activation.reshape(1, -1))
            next_layer_gradient = sigmoid_prime(val) * (w.T @ next_layer_gradient.reshape(-1, 1))

        delta_nabla_b = list(reversed(delta_nabla_b))
        delta_nabla_w = list(reversed(delta_nabla_w))

        return delta_nabla_b, delta_nabla_w

    def evaluate(self, x_test_data, y_test_data):
        # return accuracy
        return np.mean(np.argmax(self.feedforward(x_test_data), axis=0) == np.argmax(y_test_data, axis=0))

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test.T, y_test.T)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.639
Epoch: 1, Accuracy: 0.7765
Epoch: 2, Accuracy: 0.7958
Epoch: 3, Accuracy: 0.8056
Epoch: 4, Accuracy: 0.8125
Epoch: 5, Accuracy: 0.89
Epoch: 6, Accuracy: 0.9032
Epoch: 7, Accuracy: 0.909
Epoch: 8, Accuracy: 0.9111
Epoch: 9, Accuracy: 0.9133
Epoch: 10, Accuracy: 0.9154
Epoch: 11, Accuracy: 0.9177
Epoch: 12, Accuracy: 0.9199
Epoch: 13, Accuracy: 0.9209
Epoch: 14, Accuracy: 0.9222
Epoch: 15, Accuracy: 0.9231
Epoch: 16, Accuracy: 0.9239
Epoch: 17, Accuracy: 0.9243
Epoch: 18, Accuracy: 0.9253
Epoch: 19, Accuracy: 0.9261
Epoch: 20, Accuracy: 0.9266
Epoch: 21, Accuracy: 0.9267
Epoch: 22, Accuracy: 0.9274
Epoch: 23, Accuracy: 0.9278
Epoch: 24, Accuracy: 0.9284
Epoch: 25, Accuracy: 0.9282
Epoch: 26, Accuracy: 0.929
Epoch: 27, Accuracy: 0.9296
Epoch: 28, Accuracy: 0.9302
Epoch: 29, Accuracy: 0.9302
Epoch: 30, Accuracy: 0.9307
Epoch: 31, Accuracy: 0.9314
Epoch: 32, Accuracy: 0.9316
Epoch: 33, Accuracy: 0.9317
Epoch: 34, Accuracy: 0.9318
Epoch: 35, Accuracy: 0.9325
Epoch: 

In [4]:
# Exercise 2, vectorized version

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z) * (1 - sigmoid(z))

class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        # Run the network on multiple cases
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch.
        # eta is the learning rate
        delta_nabla_b, delta_nabla_w = self.backprop(x_mini_batch, y_mini_batch)
        self.weights = [w - (eta / x_mini_batch.shape[1]) * nw
                        for w, nw in zip(self.weights, delta_nabla_w)]
        self.biases = [b - (eta / x_mini_batch.shape[1]) * nb
                       for b, nb in zip(self.biases, delta_nabla_b)]

    def backprop(self, x, y):
        # For a single input (x,y) return a tuple of lists.
        # First contains gradients over biases, second over weights.

        # First initialize the list of gradient arrays
        delta_nabla_b = []
        delta_nabla_w = []

        # Then go forward remembering all values before and after activations
        # in two other array list
        a = x
        v = x
        activations = [a]
        values = [a]
        for b, w in zip(self.biases, self.weights):
            v = np.dot(w, a) + b
            values.append(v)
            a = sigmoid(v)
            activations.append(a)

        # Now go backward from the final cost applying backpropagation
        next_layer_gradient = self.cost_derivative(a, y) * sigmoid_prime(v)
        reversed_zip = list(reversed(list(zip(self.biases, self.weights, activations[:-1], values[:-1]))))
        for b, w, activation, val in reversed_zip:
            delta_nabla_b.append(np.sum(np.array(next_layer_gradient), axis=1).reshape(b.shape))
            delta_nabla_w.append((np.array(next_layer_gradient) @ activation.T).reshape(w.shape))
            next_layer_gradient = (w.T @ next_layer_gradient) * sigmoid_prime(val)

        delta_nabla_b = list(reversed(delta_nabla_b))
        delta_nabla_w = list(reversed(delta_nabla_w))

        return delta_nabla_b, delta_nabla_w

    def evaluate(self, x_test_data, y_test_data):
        # return accuracy
        return np.mean(np.argmax(self.feedforward(x_test_data), axis=0) == np.argmax(y_test_data, axis=0))

    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                self.update_mini_batch(x_mini_batch.T, y_mini_batch.T, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test.T, y_test.T)))
            else:
                print("Epoch: {0}".format(j))


network = Network([784, 30, 30, 10])
network.SGD((x_train, y_train), epochs=50, mini_batch_size=100, eta=3., test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.7839
Epoch: 1, Accuracy: 0.8506
Epoch: 2, Accuracy: 0.8779
Epoch: 3, Accuracy: 0.8906
Epoch: 4, Accuracy: 0.8994
Epoch: 5, Accuracy: 0.9053
Epoch: 6, Accuracy: 0.9112
Epoch: 7, Accuracy: 0.9145
Epoch: 8, Accuracy: 0.9168
Epoch: 9, Accuracy: 0.9213
Epoch: 10, Accuracy: 0.923
Epoch: 11, Accuracy: 0.9248
Epoch: 12, Accuracy: 0.9268
Epoch: 13, Accuracy: 0.9283
Epoch: 14, Accuracy: 0.9293
Epoch: 15, Accuracy: 0.931
Epoch: 16, Accuracy: 0.9337
Epoch: 17, Accuracy: 0.9351
Epoch: 18, Accuracy: 0.936
Epoch: 19, Accuracy: 0.9361
Epoch: 20, Accuracy: 0.936
Epoch: 21, Accuracy: 0.9369
Epoch: 22, Accuracy: 0.9374
Epoch: 23, Accuracy: 0.9383
Epoch: 24, Accuracy: 0.9387
Epoch: 25, Accuracy: 0.9397
Epoch: 26, Accuracy: 0.9403
Epoch: 27, Accuracy: 0.941
Epoch: 28, Accuracy: 0.9411
Epoch: 29, Accuracy: 0.9415
Epoch: 30, Accuracy: 0.942
Epoch: 31, Accuracy: 0.9427
Epoch: 32, Accuracy: 0.9435
Epoch: 33, Accuracy: 0.944
Epoch: 34, Accuracy: 0.9439
Epoch: 35, Accuracy: 0.9438
Epoch: 36

# Excercise 3 (optional)

Standart backpropagation method requires memorization of all outputs of all layers, which can take much of precious GPU memory.
Instead of doing that, one can memorize only a select few layers and then recompute the rest as they are needed.
Your task is to complete the code below to implement backpropagation with checkpoints.
To keep things simple, use 1-example mini-batches (or, if you are bored, vectorize the code below)

In [61]:
class NetworkWithCheckpoints(object):
    def __init__(self, sizes, checkpoints):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes) - 1
        self.sizes = sizes
        self.checkpoints = checkpoints
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        # Run the network on a single case
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def feedforward_with_checkpoints(self, x):
        # Runs network on a single case, memorizing the activations of layers included in checkpoints.
        # Notice that gs (outputs of non-linearities) are shifted by one
        fs = []
        gs = [x]
        g = x
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            f = np.dot(w, g) + b
            g = sigmoid(f)
            if i in self.checkpoints:
                fs.append(f)
                gs.append(g)
            else:
                fs.append(None)
                gs.append(None)
        return fs, gs, g


    def feedforward_between_layers(self, first_layer: int, last_layer: int, acc_f, acc_g):
        # feedforward input acc_g[first_layer] for layers [first_layer, last_layer)
        # memorizing their outputs in respective indexes of acc_f, acc_g
        g = acc_g[first_layer]
        for x in range(first_layer + 1, last_layer + 1, 1):
            f = np.dot(self.weights[x - 1], g) + self.biases[x - 1]
            g = sigmoid(f)
            acc_f[x - 1] = f
            acc_g[x] = g

        pass

    def backprop_between_layers(self, start, end, acc_f, acc_g, dLdg):
        # compute the gradients for layers [start, end)
        # dLdg is a gradient with respect to nonlinearity of layer[end-1]
        # return changed dLdG so that it is gradient with respect to nonlinearieties of layer start-1
        dLdWs = []
        dLdBs = []

        for x in reversed(range(start - 1, end - 1, 1)):
          dLdg_with_lin = sigmoid_prime(acc_f[x + 1]) * dLdg
          dLdBs.append(dLdg_with_lin.reshape(-1, 1))
          dLdWs.append(dLdg_with_lin.reshape(-1, 1) @ acc_g[x + 1].reshape(1, -1))
          dLdg = self.weights[x + 1].T @ dLdg_with_lin.reshape(-1, 1)

        return reversed(dLdWs), reversed(dLdBs), dLdg

    def update_mini_batch(self, x_mini_batch, y_mini_batch, eta):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation with checkpoints to compute the gradient.
        # For this exercise, we assume 1 element mini_batch
        # eta is the learning rate
        x_mini_batch = x_mini_batch.reshape(-1, 1)
        y_mini_batch = y_mini_batch.reshape(-1, 1)

        fs, gs, output = self.feedforward_with_checkpoints(x_mini_batch)
        dLdg = output - y_mini_batch
        for start, end in reversed(list(zip([-1] + self.checkpoints, self.checkpoints + [self.num_layers-1]))):
            # if checkpoints are (a, b) then we can backprop through layers [a+1, b] inclusive
            start += 1
            end += 1
            # those copies are inefficient, but we do them to keep indexing simple
            acc_f = fs.copy()
            acc_g = gs.copy()
            self.feedforward_between_layers(start, end, acc_f, acc_g)
            nabla_w, nabla_b, dLdg = self.backprop_between_layers(start, end, acc_f, acc_g, dLdg)
            self.weights[start:end] = [w - eta * dw for w, dw in zip(self.weights[start:end], nabla_w)]
            self.biases[start:end] = [b - eta * db for b, db in zip(self.biases[start:end], nabla_b)]


    def evaluate(self, x_test_data, y_test_data):
        # return accuracy
        return np.mean(np.argmax(self.feedforward(x_test_data), axis=0) == np.argmax(y_test_data, axis=0))

    def cost_derivative(self, output_activations, y):
        return (output_activations-y)

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                y_mini_batch = y_train[i*mini_batch_size:(i*mini_batch_size + mini_batch_size)]
                self.update_mini_batch(x_mini_batch, y_mini_batch, eta)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate(x_test.T, y_test.T)))
            else:
                print("Epoch: {0}".format(j))

In [64]:
network = NetworkWithCheckpoints([784, 30, 30, 10], checkpoints=[1])
network.SGD((x_train, y_train), epochs=10, mini_batch_size=1, eta=1., test_data=(x_test, y_test) )

Epoch: 0, Accuracy: 0.8842
Epoch: 1, Accuracy: 0.9013
Epoch: 2, Accuracy: 0.907
Epoch: 3, Accuracy: 0.8987
Epoch: 4, Accuracy: 0.9125
Epoch: 5, Accuracy: 0.9067
Epoch: 6, Accuracy: 0.9209
Epoch: 7, Accuracy: 0.9283
Epoch: 8, Accuracy: 0.9218
Epoch: 9, Accuracy: 0.9198
